In [3]:
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta

# Load project details from CSV
project_details = pd.read_csv('/content/Project Details.csv')

# Load user details from CSV
user_details = pd.read_csv('/content/User Details.csv')

# Initialize Faker to generate fake data
fake = Faker()

# Create an empty list to store allocation details
allocation_data = []

# Shuffle the order of projects and users
project_indices = list(range(len(project_details)))
random.shuffle(project_indices)

user_indices = list(range(len(user_details)))
random.shuffle(user_indices)

# Function to assign resources to projects
def assign_resources(project_name, start_date, end_date, project_id, user_name, user_email):
    allocation_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    # Generate allocation end date as 2 or 3 weeks after allocation start date
    allocation_end_date = allocation_date + timedelta(weeks=random.randint(2, 3))
    allocation_data.append({
        'projectName': project_name,
        'userName': user_name,
        'userEmail': user_email,
        'allocationStartDate': allocation_date.strftime('%Y-%m-%d'),
        'allocationEndDate': allocation_end_date.strftime('%Y-%m-%d')
    })

# Iterate over shuffled users and assign them to different projects
num_allocations = min(len(project_indices), len(user_indices))
for i in range(num_allocations):
    project_row = project_details.iloc[project_indices[i]]
    user_row = user_details.iloc[user_indices[i]]
    start_date = datetime.strptime(project_row['startDate'], '%d-%m-%Y')
    end_date = datetime.strptime(project_row['endDate'], '%d-%m-%Y')
    project_id = project_row['id']
    user_name = user_row['firstName']
    user_email = user_row['email']
    assign_resources(project_row['name'], start_date, end_date, project_id, user_name, user_email)

# Convert allocation data to DataFrame
allocation_df = pd.DataFrame(allocation_data)

# Save allocation data to CSV
allocation_df.to_csv('resource_allocations.csv', index=False)


In [2]:
%pip install Faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.9 MB/s eta 0:00:00


In [4]:
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta

# Load resource allocation data from CSV
resource_allocations = pd.read_csv('resource_allocations.csv')

# Load user details from CSV
user_details = pd.read_csv('User Details.csv')

# Initialize Faker to generate fake data
fake = Faker()

# Tasks list
tasks = [
    "Requirement Analysis", "Design", "Development", "Testing", "Bug Fixing",
    "Code Review", "Deployment", "Documentation", "Meetings", "Research and Development", "Integration"
]

# Project types list
project_types = [
    "BAU activity", "Sales activity", "Client project"
]

# Create an empty list to store fake data
fake_data = []

# Function to generate fake data for a given project allocation
def generate_fake_data(project_name, start_date, user_name, user_email, project_type):
    # Get the end date (Sunday) of the same week as the start date
    end_date = start_date + timedelta(days=6)

    # Generate total hours for the week (between 30 to 50)
    total_hours = random.randint(30, 50)

    # Generate random hours for each day while ensuring the total falls within the total_hours range
    mon_to_fri_hours = [random.randint(5, 8) for _ in range(5)]
    sat_sun_hours = [random.randint(2, 5) for _ in range(2)]

    # Adjust total hours to accommodate for weekends
    remaining_hours = total_hours - sum(mon_to_fri_hours) - sum(sat_sun_hours)

    # If remaining hours are negative, distribute evenly across weekdays and weekends
    if remaining_hours < 0:
        excess_hours = abs(remaining_hours)
        excess_per_day = excess_hours // 7
        mon_to_fri_hours = [max(0, hour - excess_per_day) for hour in mon_to_fri_hours]
        sat_sun_hours = [max(0, hour - excess_per_day) for hour in sat_sun_hours]
    # If remaining hours are positive, add them to weekdays
    else:
        mon_to_fri_hours = [hour + (remaining_hours // 5) for hour in mon_to_fri_hours]

    # Create fake data entry for each task
    for task in tasks:
        fake_entry = {
            'UID': fake.uuid4(),
            'userName': user_name,
            'email': user_email,
            'projectType': project_type,
            'projectName': project_name,
            'task': task,
            'comment': 'task1',
            'mon': mon_to_fri_hours[0],  # Assign hours for each day
            'tue': mon_to_fri_hours[1],
            'wed': mon_to_fri_hours[2],
            'thur': mon_to_fri_hours[3],
            'fri': mon_to_fri_hours[4],
            'sat': sat_sun_hours[0],
            'sun': sat_sun_hours[1],
            'total': total_hours,
            'startDate': start_date.strftime('%b %d, %Y'),
            'endDate': end_date.strftime('%b %d, %Y')
        }
        # Append the fake data entry to the list
        fake_data.append(fake_entry)

# Iterate over each allocation and generate fake data
for _, allocation_row in resource_allocations.iterrows():
    project_name = allocation_row['projectName']
    start_date = datetime.strptime(allocation_row['allocationStartDate'], '%Y-%m-%d')
    end_date = datetime.strptime(allocation_row['allocationEndDate'], '%Y-%m-%d')
    # Find the user details for the allocated project
    user_row = user_details[user_details['firstName'] == allocation_row['userName']].iloc[0]
    user_name = user_row['firstName']
    user_email = user_row['email']
    # Randomly select project type from the list
    project_type = random.choice(project_types)
    # Ensure start date falls on a Monday within the allocated start date and end date range
    start_date = start_date + timedelta(days=(7 - start_date.weekday()))
    # Ensure end date is 7 days after the start date and within the allocated range
    end_date = min(start_date + timedelta(days=6), end_date)
    generate_fake_data(project_name, start_date, user_name, user_email, project_type)

# Convert fake data to DataFrame
fake_data_df = pd.DataFrame(fake_data)

# Save fake data to CSV
fake_data_df.to_csv('Timesheet_dataset.csv', index=False)
